# Decision Tree implementation ( without using library sk_learn)

### Openning The Wine Data Set

#### Importing Required Libraries

In [104]:
import numpy as np
import pandas as pd

#### Opeening The data set

In [105]:
data=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/wine-dataset.csv")

In [106]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [107]:
data.head(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,0
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,0
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,0
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,0
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,0
5,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,0
6,6.2,0.32,0.16,7.0,0.045,30.0,136.0,0.9949,3.18,0.47,9.6,0
7,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,0
8,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,0
9,8.1,0.22,0.43,1.5,0.044,28.0,129.0,0.9938,3.22,0.45,11.0,0


In [108]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


##### Here we can verify there is not any non_null attribute



## Implementing Decision Tree

### Implementing Entropy Code

In [109]:
def entropy(col):                                     #to calculate entropy on a given attribute
    counts=np.unique(col,return_counts=True)
    N=float(col.shape[0]) #Counting total elements in array
    ent=0.0
    for ix in counts[1]:
        p=ix/N
        ent +=(-1.0 *p*np.log2(p))
    return ent

In [114]:
x=np.array([1,1,1,2,2,2])
print("Entropy for sample array [1,1,1,2,2,2]:",entropy(x))
y=np.array([1,1,1])
print(print("Entropy for sample array [1,1,1]:",entropy(y)))

Entropy for sample array [1,1,1,2,2,2]: 1.0
Entropy for sample array [1,1,1]: 0.0
None


#### Above results show correct entropy for given sample data array. So, Let's see entropy of column quality in our dataset

In [115]:
print(entropy(data['quality']))

0.7535669054779298


#### Here we can see that dataset has very high entropy

### Implementing Information Gain Using Entropy 

In [116]:
def divide(x_data,key,th_val):                 #to divide the dataset into binary unicariate split on optimal attribute
    x_left=pd.DataFrame([],columns=x_data.columns)
    x_right=pd.DataFrame([],columns=x_data.columns)
    
    for i in range(x_data.shape[0]):
        val=x_data[key].loc[i]
        if(val>th_val):
            x_right=x_right.append(x_data.loc[i])
        else:
            x_left=x_left.append(x_data.loc[i])
    print("Number of nodes in left child: ",x_left.shape[0],"Number of nodes in right child: ",x_right.shape[0])
    return x_left,x_right

In [117]:
def info_gain(x_data,key,th_val):             # to find information gain of each attribute
    print("Key: \n",key)
    left,right=divide(x_data,key,th_val)
    l=float(left.shape[0]/x_data.shape[0])
    r=float(right.shape[0]/x_data.shape[0])
    if(left.shape[0]==0 or right.shape[0]==0): return -1000000
    i_gain=entropy(x_data.quality)-(l*entropy(left.quality)+r*entropy(right.quality))
    return i_gain

In [118]:
features=['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol']                         #storing columns into list called features

###  Decision tree code

In [119]:
class DecisionTree:
    def __init__(self,depth=0,max_depth=5):
        self.left=None
        self.right=None
        self.fkey=None
        self.fval=None
        self.max_depth=max_depth
        self.depth=depth
        self.target=None
    def train(self,x_data):
        features=['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol']
        info_gains=[]
        for i in features:
            i_gain=info_gain(x_data,i,x_data[i].mean())
            info_gains.append(i_gain)
            
        self.fkey=features[np.argmax(info_gains)]
        self.fval=x_data[self.fkey].mean()
        
        print("Optimal attribute to be used to split is :",self.fkey," ",self.depth)
        
        left,right=divide(x_data,self.fkey,self.fval)
        left=left.reset_index(drop=True)
        right=right.reset_index(drop=True)
        
        
        if(left.shape[0]==0 or right.shape[0]==0):
            
            if(x_data.quality.mean()>=0.5):
                self.target=1
            else:
                self.target=0
            return
            
        if(self.depth >=self.max_depth):
            if(x_data.quality.mean()>=0.5):
                self.target=1
            else:
                self.target=0
            return
        
        self.left=DecisionTree(depth=self.depth+1,max_depth=self.max_depth)
        self.left.train(left)
        self.right=DecisionTree(depth=self.depth+1,max_depth=self.max_depth)
        self.right.train(right)
        
        if(x_data.quality.mean()>=0.5):
            self.target=1
        else:
            self.target=0
        return
    
    def predict(self,test):
        if(test[self.fkey]>=self.fval):
            if self.right is None:
                return self.target
            return self.right.predict(test)
        else:
            if self.left is None:
                return self.target
            return self.left.predict(test)

### Dividing The Data

In [121]:
count=int(.7*data.shape[0])
print("Total data rows used for training:", count)

Total data rows used for training: 3428


In [122]:
train_data=data[0:3428]

In [123]:
train_data.shape #training data size

(3428, 12)

In [125]:
test_data=data[3429:] #remaining data for testing

### Predicting The Values Using Decision Tree 

In [126]:
dt=DecisionTree()

In [127]:
dt.train(train_data)           #to train data 70% once

Key: 
 fixed acidity
Number of nodes in left child:  1938 Number of nodes in right child:  1490
Key: 
 volatile acidity
Number of nodes in left child:  1968 Number of nodes in right child:  1460
Key: 
 citric acid
Number of nodes in left child:  1979 Number of nodes in right child:  1449
Key: 
 residual sugar
Number of nodes in left child:  1925 Number of nodes in right child:  1503
Key: 
 chlorides
Number of nodes in left child:  2040 Number of nodes in right child:  1388
Key: 
 free sulfur dioxide
Number of nodes in left child:  1843 Number of nodes in right child:  1585
Key: 
 total sulfur dioxide
Number of nodes in left child:  1788 Number of nodes in right child:  1640
Key: 
 density
Number of nodes in left child:  1799 Number of nodes in right child:  1629
Key: 
 pH
Number of nodes in left child:  1820 Number of nodes in right child:  1608
Key: 
 sulphates
Number of nodes in left child:  1866 Number of nodes in right child:  1562
Key: 
 alcohol
Number of nodes in left child:  184

### Predicting The Values

In [128]:
dt.predict(test_data.loc[4004]) #to print predicted value of entered row

1

In [129]:
test_data['quality'].loc[4004]  #to print actual value of entered row

1

In [130]:
test_data1=test_data.reset_index(drop=True)

In [131]:
for i in range(test_data1.shape[0]):             #to test the remaining 30% of data
    test_data1['predicted_output']=dt.predict(test_data1.loc[i])

In [132]:
test_data1.head(10) #prints some sample predicted output

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,predicted_output
0,7.1,0.180,0.39,14.50,0.051,48.0,156.0,0.99947,3.35,0.78,9.1,0,0
1,7.1,0.170,0.40,14.55,0.047,47.0,156.0,0.99945,3.34,0.78,9.1,0,0
2,7.1,0.180,0.39,15.25,0.047,45.0,158.0,0.99946,3.34,0.77,9.1,0,0
3,7.8,0.290,0.29,3.15,0.044,41.0,117.0,0.99153,3.24,0.35,11.5,0,0
4,6.2,0.255,0.27,1.30,0.037,30.0,86.0,0.98834,3.05,0.59,12.9,1,0
5,8.2,0.340,0.29,5.20,0.076,19.0,92.0,0.99138,2.95,0.39,12.5,0,0
6,6.5,0.240,0.28,1.10,0.034,26.0,83.0,0.98928,3.25,0.33,12.3,0,0
7,6.9,0.240,0.23,7.10,0.041,20.0,97.0,0.99246,3.10,0.85,11.4,0,0
8,6.7,0.400,0.22,8.80,0.052,24.0,113.0,0.99576,3.22,0.45,9.4,0,0
9,6.7,0.300,0.44,18.50,0.057,65.0,224.0,0.99956,3.11,0.53,9.1,0,0


In [133]:
accuracy=np.sum(test_data1['quality']==test_data1['predicted_output'])/test_data1.shape[0]

In [134]:
print(accuracy)   #accuracy after 1 fold

0.7930565010211028


Here using early stopping , accuracy of code is shown above

# Cross Validation

### Considering K value as 10 we divide data in 10 parts 

In [135]:
size_of_each_part=data.shape[0]/10
print(size_of_each_part)

489.8


In [136]:
def accuracy(test_data):
    test_data1=test_data.reset_index(drop=True)
    for i in range(test_data1.shape[0]):
        test_data1['predicted_output']=dt.predict(test_data1.loc[i])
    accuracy=np.sum(test_data1['quality']==test_data1['predicted_output'])/test_data1.shape[0]
    return accuracy

In [137]:

def training_kfold(test_data,train_data,accuracy_list,count):
    test_data.reset_index(drop=True,inplace=True)
    train_data.reset_index(drop=True,inplace=True)
    print(count," "," Training Start")
    dt.train(train_data)
    print("Training Complete\n")
    accuracy_list.append(accuracy(test_data))

In [138]:
arr=[]
k=0
while k<4890:
    arr.append(k)
    k=k+489


In [139]:
print(arr)

[0, 489, 978, 1467, 1956, 2445, 2934, 3423, 3912, 4401]


In [142]:
count=0
accuracy_list=[]
for k in arr:
    test_data=data[k:k+488]
    if(k==0):
        train_data=data[k+489:]
    else:
        train_data=data[:k-1].append(data[k+490:])
    count=count+1
    training_kfold(test_data,train_data,accuracy_list,count)


Streaming output truncated to the last 5000 lines.
 chlorides
Number of nodes in left child:  71 Number of nodes in right child:  56
Key: 
 free sulfur dioxide
Number of nodes in left child:  69 Number of nodes in right child:  58
Key: 
 total sulfur dioxide
Number of nodes in left child:  69 Number of nodes in right child:  58
Key: 
 density
Number of nodes in left child:  67 Number of nodes in right child:  60
Key: 
 pH
Number of nodes in left child:  70 Number of nodes in right child:  57
Key: 
 sulphates
Number of nodes in left child:  71 Number of nodes in right child:  56
Key: 
 alcohol
Number of nodes in left child:  60 Number of nodes in right child:  67
Optimal attribute to be used to split is : alcohol   5
Number of nodes in left child:  60 Number of nodes in right child:  67
Key: 
 fixed acidity
Number of nodes in left child:  65 Number of nodes in right child:  59
Key: 
 volatile acidity
Number of nodes in left child:  73 Number of nodes in right child:  51
Key: 
 citric ac

In [143]:
accuracy_list

[0.8073770491803278,
 0.735655737704918,
 0.75,
 0.8217213114754098,
 0.8688524590163934,
 0.7520491803278688,
 0.7172131147540983,
 0.26639344262295084,
 0.8155737704918032,
 0.8299180327868853]

In [144]:
acc=np.sum(accuracy_list)/(10)  #recalculated accuracy

In [152]:
acc

0.7364754098360655

In [153]:
train_data.shape

(4407, 12)

In [154]:
train_data.head(5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,0
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,0
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,0
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,0
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,0


# Decision tree implementation ( with using library sklearn and gini index)

In [155]:
from sklearn.tree import DecisionTreeClassifier

In [156]:
sk_tree=DecisionTreeClassifier(max_depth=5,criterion="gini")

In [157]:
data.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

In [158]:
count=0
features=['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol']
accuracy_list=[]
for k in arr:
    test_data=data[k:k+488]
    if(k==0):
        train_data=data[k+489:]
    else:
        train_data=data[:k-1].append(data[k+490:])
    count=count+1
    sk_tree.fit(train_data[features],train_data['quality'])
    accuracy_list.append(sk_tree.score(test_data[features],test_data['quality']))
print("Accuracy using gini index :",np.mean(accuracy_list))

Accuracy using gini index : 0.8030737704918032
